In [1]:
import mysql.connector
import pandas as pd

In [2]:
conn = mysql.connector.connect(
    host = 'database-1.cbwuowec22t2.ap-northeast-2.rds.amazonaws.com',
    port = 3306,
    user = 'team2',
    password = "1234",
    database = 'eda_project' 
)

cur = conn.cursor(buffered=True)

In [5]:
query = """
    SELECT g.gu_name, d.dong_name, COUNT(h.hospital_name)
    FROM gu g, dong d, hospital h
    WHERE (g.gu_id = d.gu_id AND d.dong_id = h.dong_id)
    AND h.hospital_name LIKE '%소아%'
    GROUP BY d.dong_name
"""


df = pd.read_sql(query, conn)

df

/tmp/ipykernel_8407/738711711.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,gu_name,dong_name,COUNT(h.hospital_name)
0,강남구,개포동,9
1,강남구,논현동,2
2,강남구,대치동,6
3,강남구,도곡동,2
4,강남구,삼성동,2
...,...,...,...
166,중랑구,면목동,4
167,중랑구,묵동,1
168,중랑구,상봉동,3
169,중랑구,신내동,4


In [8]:
df['rank'] = df['COUNT(h.hospital_name)'].rank(ascending=False, method='min')

# 순위를 기준으로 정렬
df_desc = df.sort_values(by='rank')

# 확인
print(df_desc[['gu_name', 'dong_name','COUNT(h.hospital_name)', 'rank']])

    gu_name dong_name  COUNT(h.hospital_name)   rank
32      관악구       봉천동                      13    1.0
49      노원구       상계동                      10    2.0
130     양천구       신정동                      10    2.0
128     양천구        목동                      10    2.0
0       강남구       개포동                       9    5.0
..      ...       ...                     ...    ...
163      중구     만리동2가                       1  116.0
160     종로구       숭인동                       1  116.0
167     중랑구        묵동                       1  116.0
161     종로구       혜화동                       1  116.0
162     종로구       홍파동                       1  116.0

[171 rows x 4 columns]


In [21]:
hos_list = [(row['gu_name'], row['dong_name'],row['rank']) for _, row in df_desc.iterrows()]

In [22]:
hos_list

[('관악구', '봉천동', 1.0),
 ('노원구', '상계동', 2.0),
 ('양천구', '신정동', 2.0),
 ('양천구', '목동', 2.0),
 ('강남구', '개포동', 5.0),
 ('송파구', '가락동', 6.0),
 ('광진구', '자양동', 7.0),
 ('강서구', '화곡동', 7.0),
 ('서초구', '서초동', 7.0),
 ('영등포구', '신길동', 7.0),
 ('동대문구', '장안동', 11.0),
 ('송파구', '방이동', 11.0),
 ('동작구', '상도동', 11.0),
 ('노원구', '중계동', 11.0),
 ('동대문구', '답십리동', 11.0),
 ('관악구', '신림동', 11.0),
 ('은평구', '진관동', 11.0),
 ('서초구', '반포동', 11.0),
 ('강서구', '마곡동', 11.0),
 ('강남구', '대치동', 11.0),
 ('노원구', '공릉동', 21.0),
 ('도봉구', '방학동', 21.0),
 ('성동구', '하왕십리동', 21.0),
 ('금천구', '독산동', 21.0),
 ('마포구', '아현동', 21.0),
 ('성북구', '정릉동', 21.0),
 ('강남구', '역삼동', 21.0),
 ('금천구', '시흥동', 21.0),
 ('강동구', '고덕동', 21.0),
 ('강북구', '미아동', 21.0),
 ('송파구', '장지동', 21.0),
 ('송파구', '잠실동', 21.0),
 ('양천구', '신월동', 21.0),
 ('은평구', '응암동', 21.0),
 ('강서구', '염창동', 35.0),
 ('송파구', '문정동', 35.0),
 ('송파구', '신천동', 35.0),
 ('성동구', '행당동', 35.0),
 ('구로구', '구로동', 35.0),
 ('강동구', '강일동', 35.0),
 ('강동구', '성내동', 35.0),
 ('강동구', '암사동', 35.0),
 ('서초구', '방배동', 35.0),
 ('서대문구', '남가좌동'

In [25]:
def update_rank_dong_hos(gu, dong, rank):
    # 구 ID 가져오기
    cur.execute('SELECT gu_id FROM gu WHERE gu_name = %s', (gu,))
    gu_id = cur.fetchone()[0]

    # 동 ID 가져오기
    cur.execute('SELECT dong_id FROM dong WHERE dong_name = %s', (dong,))
    dong_id = cur.fetchone()[0]


    # gu_id에 해당하는 row가 존재하면 업데이트
    cur.execute(
        'UPDATE rank_for_dong SET rank_hospital = %s WHERE gu_id = %s AND dong_id = %s', 
        (rank, gu_id, dong_id))
    
    conn.commit()


In [26]:
for gu, dong, rank in hos_list:
    update_rank_dong_hos(gu, dong, rank)